# Inference notebook

## Imports

In [1]:
import os
import json
from os.path import join
from itertools import pairwise, product

import torch
import numpy as np
import pandas as pd
import polars as pl
from numpy import ndarray
from torch import nn, Tensor
import plotly.express as px
from pandas import DataFrame as DF
from kagglehub import competition_download, dataset_download, model_download

## Preprocessing

### Load dataset meta data

In [2]:
meta_data_path = dataset_download(
    handle="mauroabidalcarrer/prepocessed-cmi-2025",
    path="preprocessed_dataset/full_dataset_meta_data.json"
)
with open(meta_data_path, "r") as fp:
    meta_data = json.load(fp)

### Define preprocessing function

In [3]:
META_DATA_COLUMNS = [
    'row_id',
    'sequence_type',
    'sequence_id',
    'sequence_counter',
    'subject',
    'orientation',
    'behavior',
    'phase',
    'gesture',
]

def impute_missing_feature_data(df:DF) -> DF:
    feature_cols = list(set(df.columns) - set(META_DATA_COLUMNS))
    # Missing ToF values are already imputed by -1 which is inconvinient since we want all missing values to be NaN.    
    # So we replace them by NaN and then perform imputing.  
    to_replace = {col: -1.0 for col in df.columns if col.startswith("tof")}

    df.loc[:, feature_cols] = (
        df
        .loc[:, feature_cols]
        # df.replace with np.nan sets dtype to floar64 so we set it back to float32
        .replace(to_replace, value=np.nan)
        .astype("float32")
        .groupby(df["sequence_id"], observed=True, as_index=False)
        .ffill()
        .groupby(df["sequence_id"], observed=True, as_index=False)
        .bfill()
        # In case there are only nan in the column in the sequence
        .fillna(0)
    )
    return df

def agg_tof_cols_per_sensor(df:DF) -> DF:
    for tof_idx in range(1, 6):
        tof_name = f"tof_{tof_idx}"
        tof_cols = [f"{tof_name}_v{v_idx}" for v_idx in range(64)]
        if any(map(lambda col: col not in df.columns, tof_cols)):
            print(f"Some (or) all ToF {tof_idx} columns are not in the df. Maybe you already ran this cell?")
            continue
        df = (
            df
            # Need to use a dict otherwise the name of the col will be "tof_preffix" instead of the value it contains
            .assign(**{tof_name:df[tof_cols].mean(axis="columns")})
            .drop(columns=tof_cols)
        )
    return df

def length_normed_sequence_feat_arr(sequence: DF) -> ndarray:
    features = (
        sequence
        .loc[:, meta_data["feature_cols"]]
        .values
    )
    normed_sequence_len = meta_data["pad_seq_len"]
    len_diff = abs(normed_sequence_len - len(features))
    if len(features) < normed_sequence_len:
        padded_features = np.pad(
            features,
            ((len_diff // 2 + len_diff % 2, len_diff // 2), (0, 0)),
        )
        return padded_features
    elif len(features) > normed_sequence_len:
        return features[len_diff // 2:-len_diff // 2]
    else:
        return features

def preprocess_sequence(sequence_df:pl.DataFrame) -> ndarray:
    return (
        sequence_df                     
        .to_pandas()                            # Convert to pandas dataframe
        .pipe(impute_missing_feature_data)      # Impute missing data
        .pipe(agg_tof_cols_per_sensor)          # Aggregate ToF columns
        .loc[:, meta_data["feature_cols"]]
        .sub(meta_data["mean"])                 # std norm sequence features
        .div(meta_data["std"])
        .pipe(length_normed_sequence_feat_arr)  # get feature ndarray of sequence
        .T                                      # Transpose to swap channel and X dimensions
    )

## Load model

### Define model

In [4]:
class ResidualBlock(nn.Module):
    def __init__(self, in_chns:int, out_chns:int):
        super().__init__()
        self.blocks = nn.Sequential(
            nn.Conv1d(in_chns, out_chns, kernel_size=3, padding=1),
            nn.BatchNorm1d(out_chns),
            nn.ReLU(),
            nn.Conv1d(out_chns, out_chns, kernel_size=3, padding=1),
            nn.BatchNorm1d(out_chns),
        )
        if in_chns == out_chns:
            self.skip_connection = nn.Identity() 
        else:
            # TODO: set bias to False ?
            self.skip_connection = nn.Sequential(
                nn.Conv1d(in_chns, out_chns, 1),
                nn.BatchNorm1d(out_chns)
            )

    def forward(self, x:Tensor) -> Tensor:
        activaition_maps = self.skip_connection(x) + self.blocks(x)
        return nn.functional.relu(activaition_maps)

class Resnet(nn.Module):
    def __init__(
            self,
            in_channels:int,
            depth:int,
            # n_res_block_per_depth:int,
            mlp_width:int,
            n_class:int,
        ):
        super().__init__()
        chs_per_depth = [in_channels * 2 ** i for i in range(depth)]
        blocks_chns_it = pairwise(chs_per_depth)
        self.res_blocks = [ResidualBlock(in_chns, out_chns) for in_chns, out_chns in blocks_chns_it]
        self.res_blocks = nn.ModuleList(self.res_blocks)
        self.mlp_head = nn.Sequential(
            nn.LazyLinear(mlp_width),
            nn.ReLU(),
            nn.Linear(mlp_width, n_class),
            nn.Softmax(dim=1),
        )
        
        
    def forward(self, x:Tensor) -> Tensor:
        activation_maps = x
        for res_block in self.res_blocks:
            activation_maps = nn.functional.max_pool1d(res_block(activation_maps), 2)
        out = activation_maps.view(activation_maps.shape[0], -1)
        out = self.mlp_head(out)
        return out

model = Resnet(
    in_channels=17,
    depth=4,
    mlp_width=256,
    n_class=18,
)

### Load model weights

In [5]:
model_state_parent_dir = model_download("mauroabidalcarrer/cmi-resnet/pyTorch/normalize-full-dataset")
model_state_filename = os.listdir(model_state_parent_dir)[0]
model_state_path = join(model_state_parent_dir, model_state_filename)
model_weights_state_dict = torch.load(model_state_path, weights_only=True)["model"]
model.load_state_dict(model_weights_state_dict)

<All keys matched successfully>

## Perform inference

In [7]:

def predict(sequence_df:pl.DataFrame, _:pl.DataFrame) -> str:
    x = preprocess_sequence(sequence_df)
    x = torch.unsqueeze(Tensor(x), dim=0)
    y_pred = (
        model(x)
        .max(dim=1)[1]
        .numpy()
        .squeeze()
    )
    y_pred_str = meta_data["target_names"][y_pred]

    return y_pred_str

CATEGORY_COLUMNS = [
    'row_id',
    'sequence_type',
    'sequence_id',
    'subject',
    'orientation',
    'behavior',
    'phase',
    'gesture',
]

DATASET_DF_DTYPES = {
    "acc_x": "float32", "acc_y": "float32", "acc_z": "float32",
    "thm_1":"float32", "thm_2":"float32", "thm_3":"float32", "thm_4":"float32", "thm_5":"float32",
    "sequence_counter": "int32",
    **{col: "category" for col in CATEGORY_COLUMNS},
    **{f"tof_{i_1}_v{i_2}": "float32" for i_1, i_2 in product(range(1, 5), range(64))},
}

# if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
import kaggle_evaluation.cmi_inference_server
inference_server = kaggle_evaluation.cmi_inference_server.CMIInferenceServer(predict)
inference_server.serve()

# else:
#     df_path = competition_download(
#         handle="cmi-detect-behavior-with-sensor-data",
#         path="train.csv",
#     )
#     df = pd.read_csv(df_path, dtype=DATASET_DF_DTYPES)
#     for _, sequence in df.groupby("sequence_id", observed=True, as_index=False):
#         sequence = pl.DataFrame(sequence)
#         display(predict(sequence, None))
#         break